# 국민청원 데이터 가져오기


- 참고:
  - [국민청원 데이터셋 CSV](https://newhiwoong.github.io/%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90/%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90-%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B)
  - [청와대 국민청원 https://www1.president.go.kr/petitions](https://www1.president.go.kr/petitions)
  - [Using Selenium with Google Colaboratory https://darektidwell.com/using-selenium-with-google-colaboratory/](https://darektidwell.com/using-selenium-with-google-colaboratory/)
  - [https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path](https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path)
 
- 참고링크 중 "국민청원 데이터셋 CSV"의 내용을 토대로 수정하여 작성
- 1건당 1-2초(google colab에서 더 느림, 실 사용은 로컬을 권장함)

## 확인된 에러의 경우 
1. 없는 페이지
2. 관리자 차단 페이지

In [1]:
import sys

# install chromium, its driver, and selenium
if 'google.colab' in sys.modules:
    !apt-get update
    !apt install chromium-chromedriver
    !cp /usr/lib/chromium-browser/chromedriver /usr/bin
    !pip install selenium

# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import re

#진행률 바 
if 'ipykernel' in sys.modules:
    from tqdm import tqdm_notebook as tqdm
else:
    from tqdm import tqdm

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,820 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,461 kB]
Get:13 https://cloud.r-project.org/bin/linux/ubunt

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [3]:
def get_petition(wd, number):
    base_url='https://www1.president.go.kr/petitions/'
    try:
        wd.get(base_url+str(number))
        #ID가 petitionsView_title인 element가 로딩될 때 까지 2초 대기
        element = WebDriverWait(wd, 2).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'petitionsView_title')))
        #bs = BeautifulSoup(wd.page_source, 'html.parser')
        #lxml HTML 파서를 사용, html.parser보다 유연하고 빠른처리
        bs = BeautifulSoup(wd.page_source, 'lxml')

        progress = bs.find('div',{'class':'petitionsView_progress'}).get_text().strip()
        title = bs.find('h3',{'class':'petitionsView_title'}).get_text().strip()
        like = bs.find('h2',{'class':'petitionsView_count'}).get_text().strip()
        like = int(''.join(re.findall('\d+', str(like))))
        if progress == '답변완료':
            contents = bs.find_all('div',{'class':'View_write'})[1].get_text().strip()
            answer = bs.find_all('div',{'class':'pr_tk25'})[1].get_text().strip()[4:].strip()
        else:
            contents = bs.find('div',{'class':'View_write'}).get_text().strip()
            answer = ''
        info = bs.find('ul',{'class':'petitionsView_info_list'}).get_text().split('\n')
        category = info[1][4:]
        sday = info[2][4:]
        eday = info[3][4:]
        proponent = info[4][3:].split()[0]
        return {'number':number, 'progress':progress, 'title':title,  'like':like,
                'contents':contents, 'answer':answer, 'category':category,
                'sday':sday, 'eday':eday, 'proponent':proponent}
    except Exception as e:
        print(number, e)
        return {'number':number, 'progress':'error', 'title':'',  'like':0,
                'contents':str(e), 'answer':'', 'category':'',
                'sday':'', 'eday':'', 'proponent':''}

In [4]:
if __name__ == '__main__' or 'google.colab' in sys.modules:
    wd = webdriver.Chrome('chromedriver',options=options)
    print(get_petition(wd, 602925))
    print(get_petition(wd, 582349))
    wd.quit()

{'number': 602925, 'progress': '청원진행중', 'title': '저희는 1급 응급구조사 국가고시의 합격을 강탈 당했습니다.', 'like': 248, 'contents': "한국보건의료인국가시험원기관의(이하, 국시원) 부당한 시험 체계와 오류문제로 인한 피해를 호소드립니다. \n\n저희는 22년 응급구조학과 졸업 예정인 학생들입니다. 저희는 졸업 후 응급환자를 돌보는 \n소방구급대원, 병원 응급실, 산업체 위험현장, 등에서 근무하는 응급생명돌보미입니다.  \n\n하지만, 저희는 이제 그 꿈을 1년 뒤로 미뤄야 합니다\n\n저희는 이번 1급 응급구조사 필기시험에 불합격을 12월 7일 통지 받았습니다. \n하지만 저희들 대부분은 모든 학생들이 인정할 정도로 우수한 성적으로 학교 생활에 임한 성실한 학생입니다. \n\n2021년 11월 27일 토요일에 1급 응급구조사 필기 시험을 응시했습니다. \n하지만 저희가 응시한 시험은 공정하지 못했다고 확신합니다. \n\n이에, 저희는 4가지 문제점에 대하여 국민여러분께 호소합니다.\n\n1. 태블릿PC의 문제.\n이번 1급 응급구조사 시험은 태블릿 PC를 통해 시험을 진행하였습니다.(모든 보건의료인력 중 유일함) \n그런데, 시험 문제 해결에 필수적인 터치펜의 기능이 정상적으로 작동하지 아니하였습니다. \n버벅거림, 숫자 입력 시 의도하지 않은 숫자로 입력되어(8을 입력하면 3이 보이거나, 5를 8로 인식하거나, 1-2초 후 터치펜 입력이 태블릿 PC에 반영되거나), \n\n계산이 필요한 문항을 적절하게 해결하지 못하여 이러한 불편함 때문에 종이를 사용하여 계산형 문제를 해결하고자 하였으나, 감독관이 종이 제공을 일괄적으로 금지함으로써, 계산식 문제를 해결하는데 현저하게 불편함을 느껴, 시험 초반부터 학생들 멘탈에 심각한 문제가 발생했습니다,\n\n제가 본 시험장에서는 터치펜이 잘 인식되지 않아 불편을 호소하는 학생들이 많았고, \n이에 대해 학생들은 이 문제점에 이의제기를 하였지만 돌아오는 답변으

## 사용예

In [10]:
import pandas as pd


def get_p(num = 602925, cnt = 10):
    rtn = []
    wd = webdriver.Chrome('chromedriver',options=options)
    for idx, i in enumerate(tqdm(range(num-cnt, num))):
        tmp = get_petition(wd, i)
        if idx == 0:
            rtn_c = list(tmp.keys())
        rtn.append(list(tmp.values()))
        # 오래 유지될경우 느려지는 경향이 있어 중간에 초기화
        if idx % 200 == 0:
            wd.quit()
            wd = webdriver.Chrome('chromedriver',options=options)
    return (rtn_c, rtn)


c, rtn = get_p()
%time wd = webdriver.Chrome('chromedriver',options=options)
# %time rtn.append(list(get_petition(wd, 582349).values()))
# %time rtn.append(list(get_petition(wd, 10).values()))
wd.quit()
pd.DataFrame(rtn, columns=c)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 4.81 ms, sys: 5.73 ms, total: 10.5 ms
Wall time: 699 ms


,number,progress,title,like,contents,answer,category,sday,eday,proponent
0,602915,청원진행중,한국산업인력공단에서 주관한 공인중개사 및 기타 시험의 이의제기 프로세스의 실효성 확...,203,안녕하세요\n저는 2021년 32회 공인중개사 시험을 본 수험생 입니다.\n저는 이...,,육아/교육,2021-12-10,2022-01-09,naver
1,602916,청원진행중,12억 비과세 양도세 신고일 기준으로 해야합니다!,343,그동안 집값이 올라감에 따라 빠르게 취득세율을 올려주셨고 반면 양도세혜택은 계속 차...,,행정,2021-12-10,2022-01-09,naver
2,602917,청원진행중,*** ****는 포항 **공원민간개발사업의 잽도 안되는 사건,528,민간공원특례사업은 공원지역으로 묶여 있어서 장기간 재산권을 행사할 수 없었던 곳을 ...,,행정,2021-12-10,2022-01-09,naver
3,602918,청원진행중,미용실 거리제한 만들어주세요!!!!!,2391,안녕하세요. 대한민국에 이렇게 미용실이 많을줄은 꿈에도 몰랐어요.\n월세는 말없이 ...,,문화/예술/체육/언론,2021-12-10,2022-01-09,naver
4,602919,청원진행중,장애학생과 특수교사들의 인권보호를 위해 특수학급에 cctv를 설치해주세요,4045,[장애학생의 인권과 특수학교 교사의 인권을 CCTV 설치를 통해 지켜주세요]\n\n...,,인권/성평등,2021-12-10,2022-01-09,naver
5,602920,청원진행중,국가는 국민의 생명을 보호할 의무가 1순위입니다. 코로나 정책을 원점에서 재검토하여...,2631,"최근 2일 연속 코로나 신규확진자가 7,000명대 돌파에 위중증환자 800명대 돌파...",,보건복지,2021-12-10,2022-01-09,naver
6,602921,청원진행중,백신접종후 암에걸린 남편의 억울함을 풀어주세요,2021,저히 남편은 지난 8월24일 화이자 1차 백신접종후 일주일도 않되 극심한 허리통증과...,,보건복지,2021-12-10,2022-01-09,kakao
7,602922,청원진행중,뷔페식당 손실보상 연매출기준 현실화 해주세요,233,뷔페 식당을 운영합니다.\n2018년 인수받아 적자상태로 어렵게 운영하다가 지역사회...,,기타,2021-12-10,2022-01-09,naver
8,602923,청원진행중,국내 SNS와 커뮤니티에 대한 검열을 남발하는 'n번방 방지법' 개정을 촉구합니다.,16477,지난해 국회를 통과한 'n번방 방지법'으로 인해 12월 10일부터 카카오톡 등 국내...,,인권/성평등,2021-12-10,2022-01-09,naver
9,602924,청원진행중,"횡단보도 보행 중 우회전 차량 주행 사망 사고를 막기 위해, 더욱 강한 법률제정 요...",11891,안녕하세요. 12월 4일 경남 창원에서 발생한 '신호등 무시 우회전' 덤프트럭에 의...,,안전/환경,2021-12-10,2022-01-09,naver


[https://www1.president.go.kr/petitions/10](https://www1.president.go.kr/petitions/10)
- 관리자에 의해 비공개된 청원입니다

In [7]:
df = pd.DataFrame(rtn, columns=c)

In [9]:
df.to_csv('./petition.csv', encoding='utf-8', mode='w', index=False)

print('Completed..')